In [1]:
import os
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel,Field
from langchain_core.prompts import ChatMessagePromptTemplate,MessagesPlaceholder,ChatPromptTemplate
from langchain_groq import ChatGroq

In [2]:
groq_api_key = os.environ.get("GROQ_API_KEY")

In [3]:
class Person(BaseModel):
    """Information about a Person"""
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(default=None, description="The color of the peron's hair if known")
    height_in_meters: Optional[str] = Field(default=None, description="Height measured in meters")
    

In [4]:
prompt = ChatPromptTemplate.from_messages(
                          [
                              (
                                  "system",
                                  "You are an expert extraction algorithm. "
                                  "Only extract relevant information from the text. "
                                  "If you do not know the value of an attribute asked to extract, "
                                  "return null for the attribute's value.",
                              ),
                              ("human", "{text}"),
                          ]
                        )

In [5]:
llm = ChatGroq(model = "llama3-8b-8192", max_tokens=200)

In [6]:
runnable = prompt | llm.with_structured_output(schema=Person)

G:\Anaconda3\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The method `ChatGroq.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [7]:
text = " Vicky is 6 feet tall and has black hair"
runnable.invoke({"text" : text})

Person(name='Vicky', hair_color='black', height_in_meters='1.83')

Multiple entries

In [13]:
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(default=None, description="The color of the peron's hair if known")
    height_in_meters: Optional[str] = Field(default=None, description="Height measured in meters")

class Data(BaseModel):
    """Extracted data about people."""
    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [14]:
runnable = prompt | llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
runnable.invoke({"text" : text})

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])